In [6]:
%load_ext autoreload
%autoreload 2
%ls data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
README.md                multiclass_tweets.csv    tweets.json
binary_tweets.csv        multiclass_tweets.json
first_scrape.json        tweets-normalized-m.txt


In [44]:
import pandas as pd
from util import const, reader

In [8]:
# unused
from collections import namedtuple
def to_namedtuple(dict_): 
  return namedtuple(dict_.__name__, dict_.keys())

In [9]:
# expand each dictionary to columns in a DF 
def expand_to_columns(df, key):
  key = [key] if isinstance(key, str) else key
  expanded = [df[k].apply(pd.Series) for k in key]
  for k, e in zip(key, expanded):
    e.columns = ['{}_{}'.format(k, c) for c in e.columns]
  return pd.concat([df.drop(key, axis=1)] + expanded, axis=1)

In [12]:
%time df = reader.load_df(clean=False)

CPU times: user 1min 12s, sys: 1.47 s, total: 1min 14s
Wall time: 1min 16s


In [54]:
def info(df):
    print('unique body content:', len(df.body_content.unique()))
    print('unique ids:', len(df.id.unique()))
    print('unique users:', len(df.user_id.unique()))
    df.info()
    return df.head()
info(df)

unique body content: 101338
unique ids: 105391
unique users: 54898
<class 'pandas.core.frame.DataFrame'>
Int64Index: 105915 entries, 0 to 105914
Data columns (total 17 columns):
children            23086 non-null object
created_at          105915 non-null datetime64[ns]
favorite_count      105915 non-null int64
id                  105915 non-null int64
label               105915 non-null int64
media               10808 non-null object
parents             35177 non-null object
reply_count         105915 non-null int64
retweet_count       105915 non-null int64
user_handle         105915 non-null object
user_id             105915 non-null object
user_is_verified    105915 non-null bool
user_name           105855 non-null object
body_atreplies      85200 non-null object
body_content        105915 non-null object
body_hashtags       26003 non-null object
body_links          61780 non-null object
dtypes: bool(1), datetime64[ns](1), int64(5), object(10)
memory usage: 18.8+ MB


,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links
0,NaN,2016-03-25 02:09:00,0,713291759388921856,1,NaN,NaN,0,0,Timiochukwu2,1495729418,False,Oladipupo Matthew,[@business],@business A Korean-American man detained in No...,NaN,[goo.gl/fb/0KGMKc]
1,NaN,2016-03-15 08:43:00,0,709767014018056192,1,NaN,"[{'date': '2016-03-15 08:10:00', 'user_id': '3...",0,0,udarnik,61482996,False,John,"[@pammaysmcdonald, @chesterbadger3, @docrockte...",@pammaysmcdonald @chesterbadger3 @docrocktex26...,NaN,NaN
2,NaN,2016-03-22 23:55:00,0,712533209234464768,1,NaN,NaN,0,0,Sergiics,428221173,False,* Sergii *,[@el_pais],Cuando Antonio Vega voló en solitario http://...,NaN,[cultura.elpais.com/cultura/2016/0]
3,NaN,2016-03-21 13:10:00,0,712008560742367232,1,NaN,NaN,0,0,rubar1,128574093,False,Ruben Barrios,[@el_pais],Tribuna | Preso pero libre; por Mario Vargas L...,NaN,[elpais.com/elpais/2016/03]
4,NaN,2016-03-15 06:24:00,0,709732081929211904,1,NaN,NaN,0,0,avidaquer,210827608,False,Blog da @samegui,[@TIME],Game Of Thrones Actor Slams Fans After He Reve...,[#agentenao],[ift.tt/1TL5jBd]


In [60]:
def duplicated(df):
    return df[df.body_content.duplicated(keep=False)].sort_values(
        by=['body_content', 'user_id'])
duplicated(df)

,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links
77265,NaN,2016-03-21 12:46:00,1,712002771478781952,0,NaN,NaN,0,1,ZeroCheff,169010330,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
75953,NaN,2016-03-21 12:46:00,1,712002416984649728,0,NaN,NaN,0,1,ZeroCheff,356781591,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
45781,NaN,2016-03-15 05:33:00,0,709719053577953280,1,NaN,NaN,0,0,ThomasLiesener,130791667,False,Thomas Liesener,[@engadget],"Next for iRobot, a cute $199 kitchen and bath...",NaN,[engadget.com/2016/03/15/iro]
50176,NaN,2016-03-15 05:33:00,0,709719053577953280,1,NaN,NaN,0,0,ThomasLiesener,130791667,False,Thomas Liesener,[@engadget],"Next for iRobot, a cute $199 kitchen and bath...",NaN,[engadget.com/2016/03/15/iro]
101939,NaN,2016-03-21 16:52:00,16,712145011844845568,0,NaN,NaN,0,6,jaavedjaaferi,4071345918,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
102011,NaN,2016-03-21 16:52:00,16,712064455891222528,0,NaN,NaN,0,6,jaavedjaaferi,90628174,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
64128,"[{'date': '2016-03-27 05:00:00', 'user_id': '3...",2016-03-27 04:40:00,1,714057427189018624,0,NaN,NaN,2,1,crankyallAnimal,3160381099,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
64141,"[{'date': '2016-03-27 05:00:00', 'user_id': '3...",2016-03-27 04:40:00,1,714054511237603328,0,NaN,NaN,2,1,crankyallAnimal,631742193,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
74971,NaN,2016-03-15 23:19:00,0,709987425796427776,0,NaN,NaN,0,1,Isaackcheboiwo,1903978172,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]
81530,NaN,2016-03-15 23:19:00,0,709987380581761024,0,NaN,NaN,0,1,Isaackcheboiwo,3499405755,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]


In [65]:
files = const.FileManager()
%ls data

README.md                multiclass_tweets.csv    tweets.json
binary_tweets.csv        multiclass_tweets.json
first_scrape.json        tweets-normalized-m.txt


In [48]:
%time df_multi = reader.load_df_multi(clean=False)

CPU times: user 34 s, sys: 424 ms, total: 34.5 s
Wall time: 35.2 s


In [67]:
mcsv = pd.read_csv(files.data('multiclass_tweets.csv'))
print('total multiclass in original', len(mcsv))

total multiclass in original 65792


In [55]:
info(df_multi)

unique body content: 46754
unique ids: 49877
unique users: 18122
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49877 entries, 0 to 49876
Data columns (total 17 columns):
children            11678 non-null object
created_at          49877 non-null datetime64[ns]
favorite_count      49877 non-null int64
id                  49877 non-null int64
label               49877 non-null int64
media               4295 non-null object
parents             15594 non-null object
reply_count         49877 non-null int64
retweet_count       49877 non-null int64
user_handle         49877 non-null object
user_id             49877 non-null object
user_is_verified    49877 non-null bool
user_name           49858 non-null object
body_atreplies      36934 non-null object
body_content        49877 non-null object
body_hashtags       14432 non-null object
body_links          30834 non-null object
dtypes: bool(1), datetime64[ns](1), int64(5), object(10)
memory usage: 6.5+ MB


,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links
0,NaN,2016-03-27 06:25:00,0,714080928063229952,1,NaN,NaN,0,0,mhoust42,48494546,False,Just another slave,[@21WIRE],@21WIRE #FatherPed #VicarOfFiddly tune into ht...,"[#FatherPed, #VicarOfFiddly, #Middlesbrough]",[bambuser.com/channel/Joe_Pu]
1,NaN,2016-03-19 17:00:00,0,711341607187046400,0,NaN,NaN,0,0,sajuks,17727046,False,Saju K,[@2012gregg],New Study Links #Meditation To Wisdom http://o...,[#Meditation],[ow.ly/ZHxVP]
2,"[{'user_id': '2282222899', 'date': '2016-03-15...",2016-03-15 11:16:00,0,709805496253980672,0,NaN,"[{'user_id': '2282222899', 'date': '2016-03-15...",1,0,TonyJSelimi,107406730,False,Tony Jeton Selimi #FBPE,"[@AuthenticLifeCo, @2012gregg]",@AuthenticLifeCo @2012gregg check http://www.a...,NaN,[apathtowisdom.com]
3,NaN,2016-03-27 05:07:00,4,714061223671775232,1,NaN,NaN,0,15,shoestring911,176841121,False,Shoestring 9/11,"[@VanessaBeeley, @21WIRE]",Revealing new article describes shameful US & ...,NaN,[21stcenturywire.com/2016/03/26/one]
4,NaN,2016-03-26 22:35:00,0,713962579413110784,1,NaN,"[{'user_id': '16936686', 'date': '2016-03-26 2...",0,0,warrioroftrut,832861190,False,mary pay,[@21wire],@21wire Hindering off course! ISIS = US Stooge...,NaN,NaN


In [61]:
duplicated(df_multi)

,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links
21608,NaN,2016-03-21 12:46:00,1,712002771478781952,0,NaN,NaN,0,1,ZeroCheff,169010330,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
20312,NaN,2016-03-21 12:46:00,1,712002416984649728,0,NaN,NaN,0,1,ZeroCheff,356781591,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
46079,NaN,2016-03-21 16:52:00,16,712145011844845568,0,NaN,NaN,0,6,jaavedjaaferi,4071345918,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
46151,NaN,2016-03-21 16:52:00,16,712064455891222528,0,NaN,NaN,0,6,jaavedjaaferi,90628174,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
8589,"[{'user_id': '3160381099', 'date': '2016-03-27...",2016-03-27 04:40:00,1,714057427189018624,0,NaN,NaN,2,1,crankyallAnimal,3160381099,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
8601,"[{'user_id': '3160381099', 'date': '2016-03-27...",2016-03-27 04:40:00,1,714054511237603328,0,NaN,NaN,2,1,crankyallAnimal,631742193,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
19340,NaN,2016-03-15 23:19:00,0,709987425796427776,0,NaN,NaN,0,1,Isaackcheboiwo,1903978172,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]
25840,NaN,2016-03-15 23:19:00,0,709987380581761024,0,NaN,NaN,0,1,Isaackcheboiwo,3499405755,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]
27760,NaN,2016-03-22 00:47:00,0,712194680885268480,0,NaN,NaN,0,1,fontana_claudio,381728983,False,Claudio Fontana,[@RT_com],"""@RT_com: MORE: All flights from #brusselsairp...",[#brusselsairport],[on.rt.com/77nr]
27422,NaN,2016-03-22 00:47:00,0,712183798675415040,0,NaN,NaN,0,1,fontana_claudio,408626851,False,Claudio Fontana,[@RT_com],"""@RT_com: MORE: All flights from #brusselsairp...",[#brusselsairport],[on.rt.com/77nr]


In [56]:
tweet_ids_multiclass = set(df_multi.id)
tweet_ids_binary = set(df.id)
len(tweet_ids_binary.intersection(tweet_ids_multiclass))

49688

In [58]:
groupby_label = df.groupby('label')
for label, label_df in groupby_label:
  print(label, 'with', len(label_df), 'items')

0 with 50729 items
1 with 55186 items


In [59]:
groupby_label = df_multi.groupby('label')
for label, label_df in groupby_label:
  print(label, 'with', len(label_df), 'items')

0 with 42702 items
1 with 1097 items
2 with 2418 items
3 with 3660 items


In [64]:
len(df.drop_duplicates(subset='body_content'))

101338

In [68]:
len(df_multi)

49877

In [3]:
42707/49877

0.8562463660605089

In [4]:
1097/49877

0.021994105499528842

In [5]:
2418/49877

0.04847925897708363

In [6]:
3660/49877

0.07338051606953104